# Create binary mask of images from json lable file

In [ ]:
import json
import os
import numpy as np
from PIL import Image, ImageDraw

def create_binary_masks(json_file_path, images_dir, output_dir):
    """
    Convert JSON annotations to binary masks for image segmentation.
    """
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Load JSON annotations
    with open(json_file_path, 'r') as f:
        annotations = json.load(f)
    
    # Debug: Print available files in images directory
    print(f"\nFiles available in {images_dir}:")
    available_files = os.listdir(images_dir)
    for file in available_files:
        print(f"- {file}")
    print("\nStarting processing...")
    
    # Process each image
    for image_id, image_data in annotations.items():
        filename = image_data['filename']
        image_path = os.path.join(images_dir, filename)
        
        print(f"\nProcessing {filename}...")
        print(f"Looking for file at: {image_path}")
        
        # Skip if image doesn't exist
        if not os.path.exists(image_path):
            print(f"Warning: Image not found at {image_path}")
            print("Checking for case-sensitive filename...")
            
            # Try to find the file with case-insensitive matching
            found = False
            for available_file in available_files:
                if available_file.lower() == filename.lower():
                    filename = available_file
                    image_path = os.path.join(images_dir, filename)
                    found = True
                    print(f"Found matching file: {available_file}")
                    break
            
            if not found:
                print(f"No matching file found for {filename}")
                continue
        
        try:
            # Get image dimensions
            with Image.open(image_path) as img:
                width, height = img.size
                print(f"Successfully opened image. Dimensions: {width}x{height}")
            
            # Create blank mask
            mask = Image.new('L', (width, height), 0)
            draw = ImageDraw.Draw(mask)
            
            # Draw all regions
            regions = image_data.get('regions', [])
            for region in regions:
                shape_attrs = region['shape_attributes']
                
                if shape_attrs['name'] == 'polygon':
                    points = list(zip(
                        shape_attrs['all_points_x'],
                        shape_attrs['all_points_y']
                    ))
                    draw.polygon(points, fill=255)
            
            # Save the mask
            mask_filename = os.path.splitext(filename)[0] + '_mask.png'
            mask_path = os.path.join(output_dir, mask_filename)
            mask.save(mask_path)
            print(f"Successfully saved mask: {mask_path}")
            
        except Exception as e:
            print(f"Error processing {filename}: {str(e)}")
            continue

if __name__ == "__main__":
    # Update these paths to match your actual directory structure
    json_file = r"C:\Users\laxmi\OneDrive\Desktop\model_scratch\via_project_5Dec2024_1h4m_json.json"
    images_dir = r"C:\Users\laxmi\OneDrive\Desktop\model_scratch\sih1"  # Make sure this is correct
    output_dir = r"C:\Users\laxmi\OneDrive\Desktop\model_scratch\sih5"
    
    print("Starting script with:")
    print(f"JSON file: {json_file}")
    print(f"Images directory: {images_dir}")
    print(f"Output directory: {output_dir}")
    
    # Verify directories exist
    if not os.path.exists(json_file):
        print(f"Error: JSON file not found at {json_file}")
    if not os.path.exists(images_dir):
        print(f"Error: Images directory not found at {images_dir}")
    else:
        create_binary_masks(json_file, images_dir, output_dir)